# Hadar Sharon - The Battle of the Neighborhoods

### The Business Problem:

I am looking to open a high-end Asian restaurant in Tel-Aviv, Israel. I am unsure as of where I should open it, as there are a couple of specific issues that bother me:
1. I want my restaurant to stand out from the crowd, meaning that if there are many Asian restaurants in my area, it would negatively affect my business traffic and cause customer loss with fierce competition.
2. On the other hand, if my restaurant is located in a quiet and/or neglected area, or rather a very noisy and industrial area, it would not attract any customers either.
3. Moreover - Since my restaurant is going to be a high-end restaurant, I need it to be close to an affluent neighbourhood(s) so as to attract affluent customers.
4. Accessibility is still important - I'd still like my restaurant to be relatively accessible, thus not located at the edge of the city or far from main roads.

### The Data:

I am going to need data based on the city of Tel-Aviv, Israel, as well as its neighbourhoods.

Specifically, I need to be able to map out all Asian restaurants in Tel-Aviv, and be able to obtain insights about the residents of each neighbourhood - their income, their age, and lifestyle in general (as possible).

I am planning on using the Foursquare API to handle all the Geolocation data related to restaurants, and perhaps bound & classify each neighbourhood of the city using it (+ Visualization).

I am also looking to obtain a dataset with data about the residents of Tel-Aviv, whether from a local API or I will scrape entire datasets from the Internet if needed.